In [ ]:
%pip install pandas
%pip install numpy 
%pip install matplotlib
%pip install seaborn

In [2]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
from collections import Counter

In [3]:
# Carregar cada CSV limpo
df_netflix = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_netflix_clean.csv')
df_hbo = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_hbo_clean.csv')
df_apple = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/df_apple_clean.csv')
df_prime = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_prime_video_clean.csv')


print("📂 DATASETS CARREGADOS:")
print(f"Netflix: {len(df_netflix):,} registros")
print(f"HBO Max: {len(df_hbo):,} registros") 
print(f"Apple TV: {len(df_apple):,} registros")
print(f"Prime Video: {len(df_prime):,} registros")

📂 DATASETS CARREGADOS:
Netflix: 21,391 registros
HBO Max: 9,776 registros
Apple TV: 18,692 registros
Prime Video: 72,943 registros


In [4]:
# Adicionar coluna de plataforma
df_netflix['platform'] = 'Netflix'
df_hbo['platform'] = 'HBO Max'
df_apple['platform'] = 'Apple TV+'
df_prime['platform'] = 'Prime Video'

In [5]:

# ==========================================
# UNINDO OS DATASETS
# ==========================================

platforms = {
    'Netflix': df_netflix,
    'HBO Max': df_hbo, 
    'Apple TV': df_apple,
    'Prime Video': df_prime
}
# Concatenar todos os DataFrames em um único DataFrame
df_streaming = pd.concat([df_netflix, df_hbo, df_apple, df_prime], ignore_index=True)


print(f"\n🎬 DATASET UNIFICADO CRIADO:")
print(f"Total de registros: {len(df_streaming):,}")
print(f"Total de colunas: {len(df_streaming.columns)}")
print(f"Colunas: {list(df_streaming.columns)}")



🎬 DATASET UNIFICADO CRIADO:
Total de registros: 122,802
Total de colunas: 8
Colunas: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes', 'platform']


In [7]:
# ==========================================
    #Classifica conteúdo em três categorias principais:
    #- Clássico: Filmes antigos com relevância cultural/qualidade
    #- Lançamento: Conteúdo dos últimos 5 anos
    #- Catálogo: Todo o resto
# ==========================================

# Função para classificar o conteúdo
def classificar_conteudo(row):
    
    ano = row['releaseYear']
    votos = row['imdbNumVotes']
    rating = row['imdbAverageRating']
    
    # Lançamentos: últimos 5 anos
    if ano >= 2020:
        return 'Lançamento'
    
    # Clássicos: até 1999 com critérios de qualidade/relevância
    elif ano <= 1999:
        # Clássico consolidado: muitos votos + boa qualidade
        if votos >= 1000 and rating >= 6.5:
            return 'Clássico'
        # Clássico cult: poucos votos mas excelente qualidade
        elif votos >= 100 and rating >= 7.5:
            return 'Clássico'
        else:
            return 'Catálogo'
    
    # Conteúdo intermediário (2000-2019)
    else:
        return 'Catálogo'

# Aplicar classificação
df_streaming['classificacao'] = df_streaming.apply(classificar_conteudo, axis=1)


In [8]:
# Análise rápida das proporções
proporcoes = df_streaming.groupby(['platform', 'classificacao']).size().unstack(fill_value=0)
proporcoes_pct = proporcoes.div(proporcoes.sum(axis=1), axis=0) * 100

print("📊 ESTRATÉGIAS DAS PLATAFORMAS:")
print("="*35)
print(proporcoes_pct.round(1))

📊 ESTRATÉGIAS DAS PLATAFORMAS:
classificacao  Catálogo  Clássico  Lançamento
platform                                     
Apple TV+          63.2       9.5        27.3
HBO Max            51.2       8.8        40.1
Netflix            55.6       4.3        40.1
Prime Video        69.2       5.0        25.8


In [9]:
# Ranking das plataformas por foco em clássicos
print("\n🎭 RANKING - FOCO EM CLÁSSICOS:")
ranking_classicos = proporcoes_pct['Clássico'].sort_values(ascending=False)
for i, (platform, pct) in enumerate(ranking_classicos.items(), 1):
    print(f"{i}º {platform}: {pct:.1f}%")



🎭 RANKING - FOCO EM CLÁSSICOS:
1º Apple TV+: 9.5%
2º HBO Max: 8.8%
3º Prime Video: 5.0%
4º Netflix: 4.3%


In [10]:
# Ranking das plataformas por foco em lançamentos
print("\n🚀 RANKING - FOCO EM LANÇAMENTOS:")
ranking_lancamentos = proporcoes_pct['Lançamento'].sort_values(ascending=False)
for i, (platform, pct) in enumerate(ranking_lancamentos.items(), 1):
    print(f"{i}º {platform}: {pct:.1f}%")


🚀 RANKING - FOCO EM LANÇAMENTOS:
1º Netflix: 40.1%
2º HBO Max: 40.1%
3º Apple TV+: 27.3%
4º Prime Video: 25.8%
